<a href="https://colab.research.google.com/github/VivianOuou/NLP-Course/blob/main/course/en/chapter2/section2_Behind_the_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Behind the pipeline (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [28]:
!pip install datasets evaluate transformers[sentencepiece]

使用分词器进行预处理

与其他神经网络一样，Transformer模型无法直接处理原始文本，因此流程的第一步是将文本输入转换为模型可理解的数字。为此我们使用分词器（tokenizer），其核心功能包括：

分词：将输入拆分为单词、子词或符号（如标点）等基本单元（称为token）
数值映射：将每个token转换为对应的整数
附加输入：添加模型可能需要的其他辅助信息（如注意力掩码）

In [29]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [30]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [31]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


加载与使用模型

我们可以像下载分词器一样下载预训练模型。🤗 Transformers 提供了 AutoModel 类，同样包含 from_pretrained() 方法：

好的！我用一个更直观的例子来解释Transformer输出的三维向量结构，特别是"隐藏大小（Hidden Size）"这个关键维度。

---

### 以具体例子说明三维输出结构
假设我们有以下两个句子组成一个batch：
1. "I love NLP!"  
2. "Transformers are powerful."

经过分词和填充后，每个句子被转换为长度为6的token IDs（假设填充后长度统一为6）。使用一个隐藏层大小为4的微型Transformer模型（实际模型隐藏层大得多，这里简化说明）：

#### 1. 输入张量形状（模型接收的`input_ids`）
```python
shape = [batch_size, sequence_length] = [2, 6]
```
实际数值可能是：
```
[
  [101, 1045, 2293, 17953, 999, 102],  # "I love NLP!" 的token IDs
  [101, 19081, 2024, 3427, 1012, 102]  # "Transformers are powerful." 的token IDs
]
```

#### 2. 模型输出的隐藏状态（假设隐藏大小=4）
```python
shape = [batch_size, sequence_length, hidden_size] = [2, 6, 4]
```
实际输出可能类似：
```python
[
  # 第一个句子的6个token，每个token用4维向量表示
  [[0.1, 0.3, -0.2, 0.8],  # "[CLS]" token的表示
   [0.5, 0.2, 0.6, -0.1],  # "I"
   [0.3, 0.9, 0.4, 0.7],    # "love"
   [0.8, 0.5, -0.3, 0.2],   # "NLP"
   [0.2, 0.1, 0.0, 0.4],    # "!"
   [0.6, 0.3, 0.1, -0.2]],  # "[SEP]"
  
  # 第二个句子的6个token
  [[0.1, 0.3, -0.2, 0.8],   # "[CLS]"
   [0.7, 0.4, 0.9, -0.5],   # "Transformers"
   [0.2, 0.6, 0.3, 0.1],    # "are"
   [0.4, 0.8, -0.2, 0.5],   # "powerful"
   [0.3, 0.1, 0.7, 0.0],    # "."
   [0.6, 0.3, 0.1, -0.2]]   # "[SEP]"
]
```

#### 3. 为什么说"高维"？
- **隐藏大小=4**（本例简化值）：
  - 每个token被映射到4维空间的一个点（如"love" → [0.3, 0.9, 0.4, 0.7]）
  - 类似用4个特征描述一个token的语义
  
- **实际模型（如BERT-base）的隐藏大小=768**：
  - 每个token用768维向量表示
  - 相当于用768个数值特征编码一个token的上下文信息
  - 例如："bank"在"river bank"和"bank account"中会得到不同的768维向量

- **大模型（如GPT-3）的隐藏大小可达12288**：
  - 每个token的表示空间维度极高
  - 能捕获更细粒度的语义和语法特征

#### 4. 三维结构的实际意义
- **批处理维度**：同时处理多个句子（本例2个）
- **序列维度**：保留每个token的位置信息（本例每个句子6个token）
- **隐藏维度**：每个token的"知识存储空间"，维度越高表征能力越强

---

### 类比帮助理解
把Transformer输出想象成一个立方体：
```
       隐藏大小（768）
       ↑
      / \
     /   \
    ┌─────┐
    │     │ ← 一个token的表示（768个数值）
    └─────┘
   ↗
序列长度（16）
批大小（2） → 整个立方体包含 2×16×768 个数值
```

这种高维表示使得模型能区分：
- 同形异义词："苹果"（公司 vs 水果）
- 复杂语义："虽然下雨了，但我很开心"中的情感矛盾

Transformer模型通过高维隐藏表示（如768维或更高）能够区分复杂语义，这主要依赖于以下几个关键机制：

### 1. **上下文感知的动态编码**
   - **传统词向量问题**：像Word2Vec这样的静态嵌入会给"苹果"分配固定向量，无法区分"苹果手机"和"吃苹果"的不同含义。
   - **Transformer的解决方案**：
     - 通过自注意力机制，模型会根据句子上下文动态调整每个token的表示
     - 示例：
       ```python
       # "苹果"在不同语境下的向量差异
       苹果_公司 = [0.8, -0.2, 0.3, ..., 0.6]  # 与"手机""发布会"等词关联
       苹果_水果 = [0.3, 0.5, -0.7, ..., 0.1]  # 与"吃""甜""水果"等词关联
       ```
     - 余弦相似度计算可能显示这两个向量的相似度低于0.3（完全相同的向量为1.0）

### 2. **注意力机制的多层次理解**
   - **第一层注意力**（局部语法）：
     - "下雨" → "虽然"（转折关系）
     - "开心" → "但"（情感转折）
   - **深层注意力**（语义组合）：
     ```python
     # 情感分析中的矛盾语义处理
     虽然_vec = [0.2, -0.1, ..., 0.9]  # 携带转折预期
     下雨_vec = [0.7, -0.8, ..., -0.5] # 负面情感倾向
     但_vec   = [-0.3, 0.6, ..., 0.4]  # 强转折信号
     开心_vec = [-0.9, 0.7, ..., 0.8]  # 正面情感
     
     # 模型通过注意力权重组合：
     最终表示 = 0.3*虽然_vec + 0.1*下雨_vec + 0.4*但_vec + 0.9*开心_vec
     ```

### 3. **高维空间的几何特性**
   - **表征能力**：
     - 768维空间可以构造10^300+个不同的超平面（决策边界）
     - 相比之下，50维Word2Vec只能构造约10^15个超平面
   - **语义拓扑结构**：
     ```
     高维空间中：
     "苹果公司" ——靠近——> "科技"/"手机"
                   ↑
                  (正交轴)
                   ↓
     "苹果水果" ——靠近——> "食物"/"健康"
     ```

### 4. **层级特征提取**
   - **底层（靠近输入层）**：
     - 识别词性/基本语法（"下雨"是动词，"开心"是形容词）
   - **中层**：
     - 捕捉短语级语义（"下雨了"→负面，"很开心"→正面）
   - **高层（靠近输出层）**：
     - 构建句子级理解（转折关系的整体情感倾向）

### 5. **具体案例分析
#### 案例1：同形异义词区分
   ```python
   # 输入句子1："苹果发布了新手机"
   [CLS] 苹果 发布 了 新 手机 [SEP]
   
   # 输入句子2："我买了一个苹果"
   [CLS] 我 买 了 一 个 苹果 [SEP]
   
   # 模型处理：
   1. "苹果"的初始嵌入相同（相同的token ID）
   2. 经过多层Transformer后：
      - 句子1中的"苹果"受到"发布""手机"等词的影响→向量偏向科技公司
      - 句子2中的"苹果"受到"买""个"等词的影响→向量偏向水果
   3. 最终两者的余弦相似度可能＜0.4
   ```

#### 案例2：情感矛盾解析
   ```python
   # 输入句子："虽然下雨了，但我很开心"
   [CLS] 虽然 下雨 了 ， 但 我 很 开心 [SEP]
   
   # 关键步骤：
   1. 注意力头1发现："虽然"↔"但"（转折关系，权重0.8）
   2. 注意力头2发现："下雨"↔"不开心"（隐含关联，权重0.6）
   3. 注意力头3发现："很开心"↔"但"（情感强化，权重0.9）
   4. 最终[CLS]位置汇集所有信息：
      - 综合得分：正面情感0.7，负面情感0.3
      - 分类结果：正面（尽管含有负面词汇）
   ```

### 为什么低维表示无法做到？
假设只用20维向量：
- 无法同时编码语法/情感/指代等多层次信息
- 语义空间过于拥挤，不同含义的"苹果"向量会重叠
- 难以建立复杂的转折关系（需要更高维度的正交基）

而768+维空间可以：
- 为每个语义维度分配独立的"子空间"
- 通过线性变换构建数百万个潜在的特征组合
- 保持不同含义向量的近似正交性（互不干扰）

这种高维动态编码，正是Transformer理解复杂语言现象的核心优势。

In [32]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

In [33]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


In [34]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [35]:
print(outputs.logits.shape)

torch.Size([2, 2])


In [36]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


### 模型输出的后处理步骤解析

当模型直接输出原始结果时，我们需要通过后处理使其具有可解释性。以下是关键步骤的详细说明：

#### 1. 理解原始输出（Logits）
模型最后一层输出的原始分数称为**logits**：
```python
tensor([[-1.5607,  1.6123],  # 第一个句子的logits
        [ 4.1692, -3.3464]]) # 第二个句子的logits
```
- 这些数值没有概率意义
- 正/负值仅表示相对置信度（如第一个句子中1.6123 > -1.5607，倾向POSITIVE）

#### 2. 转换为概率（SoftMax处理）
通过SoftMax函数将logits转换为概率分布：
```python
import torch
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
```
输出结果：
```python
tensor([[0.0402, 0.9598],  # 第一个句子的概率
        [0.9995, 0.0005]]) # 第二个句子的概率
```
- `dim=-1` 表示对最后一个维度（标签维度）做归一化
- 每行两个值的和为1（符合概率分布特性）

#### 3. 标签映射
通过模型配置查看标签对应关系：
```python
model.config.id2label  # 输出: {0: 'NEGATIVE', 1: 'POSITIVE'}
```
最终预测结果：
- **第一个句子**："I've been waiting..."  
  → POSITIVE (95.98%置信度)  
- **第二个句子**："I hate this..."  
  → NEGATIVE (99.95%置信度)

#### 技术细节说明
| 步骤 | 输入 | 操作 | 输出 | 目的 |
|------|------|------|------|------|
| 模型原始输出 | 文本特征 | 线性层 | Logits | 获得原始分数 |
| SoftMax | Logits | e^x/sum(e^x) | 概率 | 数值归一化 |
| 标签映射 | 概率 | id2label | 标签 | 人类可读结果 |

#### 为什么使用Logits而非直接输出概率？
1. **训练效率**：  
   交叉熵损失函数会合并SoftMax计算，减少数值计算步骤
2. **数值稳定性**：  
   在反向传播时直接处理logits可避免梯度消失问题
3. **灵活性**：  
   方便后续接不同的损失函数（如带权重的交叉熵）

#### 可视化理解
```
原始文本 → [Tokenization] → 模型处理 → Logits → SoftMax → 概率 → 标签映射
           (预处理)        (前向传播)  ([-1.56,1.61]) → [0.04,0.96] → "POSITIVE"
```

通过这三个步骤，我们完整复现了pipeline的工作流程。这种模块化设计既保证了灵活性（可单独调整任一步骤），又确保了结果的可解释性。

In [37]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [38]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}